In [1]:
# read 11 yaml file of reactions, process to compare with 
#   corresponding reference data at T=0
#   as done in the published paper
import yaml, sys, os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize 
sys.path.insert(0, '../../karlib')
import chem_subs as chem
import multirx_subs as mrx

In [2]:
x = mrx.read_yaml('reactions.yml')
target = x['target']
# coefficients are negative for reactants and positive for products
rxns = x['reactions']

In [3]:
print(f'{len(rxns)} reactions for {target}')
moldata, Geom = mrx.read_educt_data(rxns)
oklist = mrx.check_reactions_balance(rxns, Geom)

15 reactions for dioxolane


In [4]:
T = 0.  # temperature; choose 0 or 298 (meaning 298.15)
exptl = mrx.select_expt(moldata, T)
eq6sum, uexp = mrx.eq6_sums(rxns, target, exptl)

In [5]:
# get theoretical energies for molecules
for molec, molecdata in moldata.items():
    #print(molec)
    mrx.thermo_functions(molecdata, T)
    mrx.compute_E0(molecdata)

*** Warning: "No SOC data for open-shell linear molecule dioxygen" in compute_E0()
*** Warning: "No SOC data for open-shell atom hydrogen atom" in compute_E0()


In [6]:
# use theoretical data to compute reaction enthalpies
Hrx, Srx = mrx.eq5_sums(rxns, target, moldata)
# enthalpy of formation of target
eof = [s5 - s6 for s5, s6 in zip(Hrx, eq6sum)]

In [7]:
# make table of results
print(f'Results for {target} at T = {T}')
dfrx = pd.DataFrame({f'EoF': eof, 'dH(rxn)': Hrx, 'uexp': uexp,
                     'Reaction': [mrx.reaction_string(rxn) for rxn in rxns]})
# get values of rho variants (list of tuples)
rho_E, rho_T, rho_c = mrx.gather_rho(rxns, target, moldata)
dfrx['rho_E'] = rho_E
dfrx['rho_T'] = rho_T
dfrx['rho_c'] = rho_c
dfrx = dfrx.sort_values('EoF')
fmt = {col: '{:.1f}' for col in dfrx.columns}
fmt['Reaction'] = '{:s}'
display(dfrx.style.format(fmt))

Results for dioxolane at T = 0.0


,EoF,dH(rxn),uexp,Reaction,rho_E,rho_T,rho_c
0,-281.9,-813.5,0.3,dioxolane + 3 n2 = 3 hcn + 2 no + nh3,860.0,9.0,18.2
2,-280.8,396.7,0.2,dioxolane + 5 h2 = 3 ch4 + 2 h2o,484.4,15.1,39.2
1,-279.0,-368.3,0.8,dioxolane + 5 hcl = 3 ch3cl + 2 hocl,333.7,10.0,43.5
3,-278.7,115.5,0.6,dioxolane = meoac,106.7,2.6,10.7
4,-278.0,1618.1,0.1,dioxolane + 3.5 o2 = 3 co2 + 3 h2o,1600.4,2.3,34.4
6,-275.7,-130.2,0.4,dioxolane = ch2o + oxirane,153.5,1.0,14.1
8,-275.1,100.5,0.6,dioxolane + acetone = EC + c3h8,105.6,1.2,0.0
9,-274.8,-63.4,1.0,dioxolane + 2 c3h8 = cyc5 + 2 DME,62.7,0.0,4.7
7,-273.8,-196.6,0.3,dioxolane + h2o = 3 ch2o + h2,260.4,2.9,51.5
10,-273.1,785.1,0.1,dioxolane + 2 o2 = 3 co + 3 h2o,729.8,13.2,114.8


In [8]:
rho = 'rho_E'
print(f'Target = {target}')
print(f'Non-uniform weighting using {rho}:')
wmean, semw, rwmsx, a, b, change, niter = mrx.nonuniform_weighting(target, rho, dfrx,
                                                        verbose=False)
# combine SEMw with u_exp
print('\tSEMw = {:.1f}, uexp = {:.1f} kJ/mol'.format(semw, rwmsx))
unc = np.sqrt(semw**2 + rwmsx**2)  # 'rwmsx' from eq. (13)
print('\tEoF({:.1f} K) = ({:.1f} ± {:.1f}) kJ/mol (standard uncertainty)'.format(T, wmean, unc))

print('Maximum Likelihood method:')
parms, bstrap = mrx.probabilistic_weighting(target, rho, [wmean, a, b], dfrx, verbose=False)
[H, sigma0, beta] = parms
[stdH, Hvals, ttime] = bstrap
print('\tEoF = ({:.1f} ± {:.1f}) kJ/mol'.format(H, stdH))
print('\tParameter bootstrapping took {:.0f} s to compute {:d} values for H\''.format(ttime, len(Hvals)))

Target = dioxolane
Non-uniform weighting using rho_E:
	SEMw = 1.3, uexp = 0.6 kJ/mol
	EoF(0.0 K) = (-274.0 ± 1.4) kJ/mol (standard uncertainty)
Maximum Likelihood method:
	EoF = (-274.5 ± 1.1) kJ/mol
	Parameter bootstrapping took 40 s to compute 924 values for H'


In [9]:
df3, fmt = mrx.table3(target, dfrx)

In [10]:
display(df3.style.format(fmt))

,$\rho$ choice,weighting,$a$ or ${\sigma}_0$,$b$ or $\beta$,$H$,$u$($H$)
0,rho_T,nonuniform,2.80,0.3772,-274.4,1.3
1,rho_T,probabilistic,2.01,0.5574,-275.0,0.9
2,rho_c,nonuniform,6.52,0.0217,-273.7,1.9
3,rho_c,probabilistic,3.11,0.0723,-275.8,1.1
4,rho_E,nonuniform,2.80,0.0062,-274.0,1.4
5,rho_E,probabilistic,2.68,0.0090,-274.5,1.1


In [12]:
mrx.compare_refdata(target, exptl, df3, fmt, verbose=True)
display(df3.style.format(fmt))

Comparing with reference data for dioxolane:
	EoF     : -275.0
	unc     : 0.7
	unit    : kJ/mol
	source  : Local


,$\rho$ choice,weighting,$a$ or ${\sigma}_0$,$b$ or $\beta$,$H$,$u$($H$),$u_c$,error
0,rho_T,nonuniform,2.80,0.3772,-274.4,1.3,1.5,0.6
1,rho_T,probabilistic,2.01,0.5574,-275.0,0.9,1.1,-0.0
2,rho_c,nonuniform,6.52,0.0217,-273.7,1.9,2.1,1.3
3,rho_c,probabilistic,3.11,0.0723,-275.8,1.1,1.3,-0.8
4,rho_E,nonuniform,2.80,0.0062,-274.0,1.4,1.6,1.0
5,rho_E,probabilistic,2.68,0.0090,-274.5,1.1,1.3,0.5
